In [559]:

import os
import json
from Levenshtein import distance
peopleDir = "/Users/amer_/Documents/Obsidian Vault/Personal/People"
with open('testJSONS/test2.json', 'r') as file:
    json_data = file.read()
    file.close()
json_data=json.loads(json_data)
json_data
safemode = True
if safemode == True:
    print("safemode is on")
    print("this will not write to the files")
attribute_classifier = {"Likes and Dislikes":"entry","Important Details":"entry","Gift Ideas":"entry","birthday":"property","Dietary Restrictions":"property","pronouns":"property"}



safemode is on
this will not write to the files


In [560]:
def calculate_similarity(arr, string):
    string = string.lower()
    # first look for exact matches, do any of the strings in the array match the string exactly?
    def simpleSearch(arr, string):
        if string.lower() in arr:
            return [string]
        # if not, do any of the strings contain the string?
        if any(string in s.lower() for s in arr):
            return [s for s in arr if string in s]
        else:
            return None
    simpleResult = simpleSearch(arr, string)
    if(simpleResult!=None):
        return simpleResult
    
    
    
    # if not, calculate the levenshtein distance between the string and each string in the array
    # if the input string as an i, swap it for an ee and check again
    # if the input string has an ee, swap it for an i and check again
    elif 'i' in string :
        PhoneticString = string.replace('i','ee')

        print("phoneticstring",PhoneticString)
        print(arr)
        simpleResult= simpleSearch(arr, PhoneticString)
        if(simpleResult!=None):
            return simpleResult
    
    # the only way we'd get here is if the other conditions weren't met
    # only check for first names, which means split them by space and only check the first element
    

    threshold = 3
    similar_strings = []
    
    for name in arr:
        name = name.split(" ")[0]
        score = distance(name, string.lower())
        if score <= threshold:
            similar_strings.append([name, score])



    # pick whatever one in the group of similar strings has the lowest levenshtein distance
    if len(similar_strings) > 0:
        min_score = min([x[1] for x in similar_strings])
        similar_strings = [x[0] for x in similar_strings if abs(x[1] - min_score)<2]
        result =  similar_strings
        # check which one has the same first letter
        if len(similar_strings) > 1:
            print("several strings had the same distance")
            result = [x for x in similar_strings if x[0].lower() == string[0].lower()]
            if len(result) > 1:
                print("several strings had the same first letter")
                resultLower = [x for x in result if x[-1] == string[-1].lower()]
                if(len(resultLower)==0):
                    return result
                return result
            else:
                return result

        else:
            return result
    else:
        return []


In [561]:
def propertyHandle():
    print('HEYOOOO')

In [562]:
# the below function splits up the body of the note into blocks of increasing resolution, to find the block that contains the information we're looking for
def entriesHandler(body):
    
    activeBlock = body
    for i in json_data['location']:
        # split the file by the number of hashtags
        hashtags = "\n"+(int(i)+1)*"#"+" "
        activeBlock = activeBlock.split(hashtags)
        # find which block starts with the name of the location in the first line
        for block in activeBlock:
            if json_data['location'][i] == block.split("\n")[0]:
                activeBlock = block
                break
            if(int(i)>0):
                oldTitleCondition = json_data['location'][str(int(i)-1)] not in block.split("\n")[0]
            else:
                oldTitleCondition = True
            # this checks if the new block has accidentally held the title of the parent block, True if it hasn't, False if it does
            # Ex: the subheading name "Likes" is in the Heading "Likes and Dislikes", so we need to make sure we select blocks that dont have the parent

            if json_data['location'][i] in block.split("\n")[0] and oldTitleCondition:
                activeBlock = block
                break
    # if the type is an array, get the zeroth index
    if type(activeBlock) == list:
        activeBlock = activeBlock[0]

    return activeBlock
        


In [563]:
def memory(name = 'guessed_name', body = 'body',properties='properties',personalFile = ''):
    firstLocation = json_data['location']['0']

    if(attribute_classifier[firstLocation]=="entry"):
        highlightedSection = entriesHandler(body)
        print(highlightedSection)
        updatedSection = highlightedSection + "\n- " + json_data['item']
        updatedFile = personalFile.replace(highlightedSection, updatedSection)
        # write this to a file
        if(safemode):
            newFileDir = peopleDir + "/" + name + "1"+".md"
        else:
            newFileDir = peopleDir + "/" + name +".md"
        with open(newFileDir, 'w') as file:
            file.write(updatedFile)
            file.close()
    elif (attribute_classifier[firstLocation]=="property"):
        propertyHandle(properties)

In [564]:
def retrieval(body = 'body',properties='properties'):
    firstLocation = json_data['location']['0']
    if(attribute_classifier[firstLocation]=="entry"):
        relevantSection = entriesHandler(body)
    elif (attribute_classifier[firstLocation]=="property"):
        propertyHandle(properties)
    print(relevantSection)



In [565]:
# print all the files in the people directory
people = []
for file in os.listdir(peopleDir):
    people.append(file[:-3])
name = json_data['person']
# Perform soft similarity search
guessed_name = ""

try:
    # lowercase the array
    peopleLower = [x.lower() for x in people]
    guessed_name = (calculate_similarity(peopleLower, json_data["person"])[0])
    guessed_name = (people[peopleLower.index(guessed_name)])
    # this then returns the closest first name within our list to the query, allegedly the 
except Exception as e:
    print(e)
    print("no match found")


# from here, we want to load the file with the name that was guessed
personalFileDir = peopleDir + "/" + guessed_name + ".md"
with open(personalFileDir, 'r') as file:
    personalFile = file.read()
    file.close()


# there are two ways to treat a given piece of information, we want to classify something as either a property, or an entry
# a property will be treated by being put in the relevant property section, ex. birthday will be in the property, allergy is a property, etc.
# an entry will be treated by being put in the relevant entry section


personalFile
# first we should split off the properties list, which is the first thing in the file, it will be split from the second ---
properties,body = personalFile.split("---")[1:3]
warning = '''
> [!warning]-
> ## Important Reminders
> - **Do Not Tag with Arguments:** Avoid memorializing arguments or conflicts.
> - **Only Factual Information:** Record only freely given information, no Googling.
> - **Comfortable Tagging:** Tag people to observe their influence but also remember happy memories.
> - **Initial Population:** Fill out information for close friends, optionally using contacts.
> 
> '''
body = body.replace(warning,"")
# print(entries)
if json_data["type"] == "memory":
    print(guessed_name)
    memory(name=guessed_name,body=body,properties = properties,personalFile = personalFile)
elif json_data["type"] == "retrieval":
    retrieval(body=body,properties = properties)

**Likes:**
- [[Beebadoobe]], the singer
- [[../Day to Day Interactions/Restaurants/Mendocino Farms|Mendocino Farms]]


In [ ]:
# Arbitrary change